# Benchmark

Problems selector

In [1]:
using EO
using FileIO, JLD2

"""simple problems"""
simple_problems = EO.setup_simple_problems() # definitions of problems g06,g08,g11,g24
#usage: g6 objective = simple_problems[1].objecitve, g6 constraints = simple_problems[1].constraints

"""unsimple problems"""
unsimple_problems = EO.setup_unsimple_problems() # definitions of problems g04, g05, g09 and g21.

unsimple_problems

Stochastic Ranking

In [3]:
for j in 1:4
    problem_set = simple_problems
    problem_id = j 
    constraints = problem_set[problem_id].constraints
    pop_size = 100
    dimension = problem_set[problem_id].dimension
    
    objective_function  = problem_set[problem_id].objective
    initialization      = enclose_noargs(interval_real_initialization, dimension, pop_size, objective_function, 0, 100)
    selection           = enclose_arguments(EO.s_stochastic_tournament, 30, 3, constraints, 0.4)
    crossover           = enclose_arguments(EO.cr_parent_sum, pop_size)
    mutation            = enclose_arguments(gaussian_preturbation!, 0.25)
    replacement         = EO.enclose_replacement(EO.r_keep_best_n_stoch, pop_size, constraints, 0.4)
    termination         = enclose_argument(iteration_termination, pop_size*100)
    
    for i in 1:100
        solution = solvink_hart(objective_function, initialization, selection, crossover, mutation, replacement, termination; constraints=constraints)
        save(joinpath(@__DIR__, "benchmarks", "constr", "stoch", "ez", "problem_"*string(j)*"_"*string(i)*".jld2"), "solution", solution)
    end 
end


In [13]:
for j in 1:4
    problem_set = unsimple_problems
    problem_id = j 
    constraints = problem_set[problem_id].constraints
    pop_size = 100
    dimension = problem_set[problem_id].dimension
    
    objective_function  = problem_set[problem_id].objective
    initialization      = enclose_noargs(interval_real_initialization, dimension, pop_size, objective_function, 0, 100)
    selection           = enclose_arguments(EO.s_stochastic_tournament, 30, 3, constraints, 0.4)
    crossover           = enclose_arguments(EO.cr_parent_sum, pop_size)
    mutation            = enclose_arguments(gaussian_preturbation!, 0.25)
    replacement         = EO.enclose_replacement(EO.r_keep_best_n_stoch, pop_size, constraints, 0.4)
    termination         = enclose_argument(iteration_termination, pop_size*100)
    
    for i in 1:100
        solution = solvink_hart(objective_function, initialization, selection, crossover, mutation, replacement, termination; constraints=constraints)
        save(joinpath(@__DIR__, "benchmarks", "constr", "stoch", "hard", "problem_"*string(j)*"_"*string(i)*".jld2"), "solution", solution)
    end 
end

MOEA

In [2]:
for j in 1:4
    problem_set = simple_problems
    problem_id = j 
    constraints = problem_set[problem_id].constraints
    pop_size = 100
    dimension = problem_set[problem_id].dimension
    
    f_pens(x::RealChromosome, constraints::Vector{EO.Constraint}) = sum(EO.Gs(x, constraints))

    F = typeof(enclose_arguments(EO.f_weighted_sum, ones(dimension)))
    objective_function  = EO.MultiObjFunction{F}([problem_set[problem_id].objective, enclose_arguments(f_pens, constraints)])
    initialization      = enclose_noargs(interval_real_initialization, dimension, pop_size, objective_function, 0, 100)
    selection           = enclose_arguments(EO.s_tournament, 30, 5)
    crossover           = enclose_arguments(EO.cr_parent_sum, pop_size)
    mutation            = enclose_arguments(gaussian_preturbation!, 0.25)
    replacement         = EO.enclose_replacement(EO.r_NSGA, pop_size)
    termination         = enclose_argument(iteration_termination, pop_size*100)
    
    for i in 1:100
        solution = solvink_hart(objective_function, initialization, selection, crossover, mutation, replacement, termination)
        save(joinpath(@__DIR__, "benchmarks", "constr", "MOEA", "ez", "problem_"*string(j)*"_"*string(i)*".jld2"), "solution", solution)
    end 
end

In [2]:
for j in 3:4
    problem_set = unsimple_problems
    problem_id = j 
    constraints = problem_set[problem_id].constraints
    pop_size = 100
    dimension = problem_set[problem_id].dimension
    
    f_pens(x::RealChromosome, constraints::Vector{EO.Constraint}) = sum(EO.Gs(x, constraints))

    F = typeof(enclose_arguments(EO.f_weighted_sum, ones(dimension)))
    objective_function  = EO.MultiObjFunction{F}([problem_set[problem_id].objective, enclose_arguments(f_pens, constraints)])
    initialization      = enclose_noargs(interval_real_initialization, dimension, pop_size, objective_function, 0, 100)
    selection           = enclose_arguments(EO.s_tournament, 30, 5)
    crossover           = enclose_arguments(EO.cr_parent_sum, pop_size)
    mutation            = enclose_arguments(gaussian_preturbation!, 0.25)
    replacement         = EO.enclose_replacement(EO.r_NSGA, pop_size)
    termination         = enclose_argument(iteration_termination, pop_size*100)
    
    for i in 1:100
        solution = solvink_hart(objective_function, initialization, selection, crossover, mutation, replacement, termination)
        save(joinpath(@__DIR__, "benchmarks", "constr", "MOEA", "hard", "problem_"*string(j)*"_"*string(i)*".jld2"), "solution", solution)
    end 
end

adaptive penalization

In [2]:
for j in 1:4
    problem_set = simple_problems
    problem_id = j 
    constraints = problem_set[problem_id].constraints
    penatly = EO.p_non_linear   # rg*sum(G(constraints)).^2
    pop_size = 100
    dimension = problem_set[problem_id].dimension
    
    objective_function  = problem_set[problem_id].objective
    initialization      = enclose_noargs(interval_real_initialization, dimension, pop_size, objective_function, 0, 100)
    selection           = enclose_arguments(EO.s_tournament, 30, 3)
    crossover           = enclose_arguments(EO.cr_parent_sum, pop_size)
    mutation            = enclose_arguments(gaussian_preturbation!, 0.25)
    replacement         = EO.enclose_replacement(EO.r_keep_best_n, pop_size)
    termination         = enclose_argument(iteration_termination, pop_size*100)
    
    for i in 1:100
        solution = solvink_hart(objective_function, initialization, selection, crossover, mutation, replacement, termination; constraints=constraints, penalty=penatly)
        save(joinpath(@__DIR__, "benchmarks", "constr", "rg", "ez", "problem_"*string(j)*"_"*string(i)*".jld2"), "solution", solution)
    end 
end

In [4]:
for j in 1:4
    problem_set = unsimple_problems
    problem_id = j 
    constraints = problem_set[problem_id].constraints
    penatly = EO.p_non_linear   # rg*sum(G(constraints)).^2
    pop_size = 100
    dimension = problem_set[problem_id].dimension
    
    objective_function  = problem_set[problem_id].objective
    initialization      = enclose_noargs(interval_real_initialization, dimension, pop_size, objective_function, 0, 100)
    selection           = enclose_arguments(EO.s_tournament, 30, 3)
    crossover           = enclose_arguments(EO.cr_parent_sum, pop_size)
    mutation            = enclose_arguments(gaussian_preturbation!, 0.25)
    replacement         = EO.enclose_replacement(EO.r_keep_best_n, pop_size)
    termination         = enclose_argument(iteration_termination, pop_size*100)
    
    for i in 1:100
        solution = solvink_hart(objective_function, initialization, selection, crossover, mutation, replacement, termination; constraints=constraints, penalty=penatly)
        save(joinpath(@__DIR__, "benchmarks", "constr", "rg", "hard", "problem_"*string(j)*"_"*string(i)*".jld2"), "solution", solution)
    end 
end

adaptive penalization, vectorized rg

In [5]:
for j in 1:4
    problem_set = simple_problems
    problem_id = j 
    constraints = problem_set[problem_id].constraints
    penatly = EO.p_non_linear   # rg*sum(G(constraints)).^2
    pop_size = 100
    dimension = problem_set[problem_id].dimension
    
    objective_function  = problem_set[problem_id].objective
    initialization      = enclose_noargs(interval_real_initialization, dimension, pop_size, objective_function, 0, 100)
    selection           = enclose_arguments(EO.s_tournament, 30, 3)
    crossover           = enclose_arguments(EO.cr_parent_sum, pop_size)
    mutation            = enclose_arguments(gaussian_preturbation!, 0.25)
    replacement         = EO.enclose_replacement(EO.r_keep_best_n, pop_size)
    termination         = enclose_argument(iteration_termination, pop_size*100)
    
    for i in 1:100
        solution = solvink_hart(objective_function, initialization, selection, crossover, mutation, replacement, termination, "vec"; constraints=constraints, penalty=penatly)
        save(joinpath(@__DIR__, "benchmarks", "constr", "rg_vec", "ez", "problem_"*string(j)*"_"*string(i)*".jld2"), "solution", solution)
    end
end

In [6]:
for j in 1:4
    problem_set = unsimple_problems
    problem_id = j 
    constraints = problem_set[problem_id].constraints
    penatly = EO.p_non_linear   # rg*sum(G(constraints)).^2
    pop_size = 100
    dimension = problem_set[problem_id].dimension
    
    objective_function  = problem_set[problem_id].objective
    initialization      = enclose_noargs(interval_real_initialization, dimension, pop_size, objective_function, 0, 100)
    selection           = enclose_arguments(EO.s_tournament, 30, 3)
    crossover           = enclose_arguments(EO.cr_parent_sum, pop_size)
    mutation            = enclose_arguments(gaussian_preturbation!, 0.25)
    replacement         = EO.enclose_replacement(EO.r_keep_best_n, pop_size)
    termination         = enclose_argument(iteration_termination, pop_size*100)
    
    for i in 1:100
        solution = solvink_hart(objective_function, initialization, selection, crossover, mutation, replacement, termination, "vec"; constraints=constraints, penalty=penatly)
        save(joinpath(@__DIR__, "benchmarks", "constr", "rg_vec", "hard", "problem_"*string(j)*"_"*string(i)*".jld2"), "solution", solution)
    end 
end

MMOEA

In [3]:
for j in 1:4
    problem_set = simple_problems
    problem_id = j 
    constraints = problem_set[problem_id].constraints
    pop_size = 100
    dimension = problem_set[problem_id].dimension

    f_pens(x::RealChromosome, constraints::Vector{EO.Constraint}) = sum(EO.Gs(x, constraints))
    objectives = Vector{Any}([problem_set[problem_id].objective])
    for constraint in constraints
        push!(objectives, constraint)
    end
    
    F = typeof(f_pens)
    objective_function  = EO.MultiObjFunction{F}(objectives)
    initialization      = enclose_noargs(interval_real_initialization, dimension, pop_size, objective_function, 0, 100)
    selection           = enclose_arguments(EO.s_tournament, 30, 3)
    crossover           = enclose_arguments(EO.cr_parent_sum, pop_size)
    mutation            = enclose_arguments(gaussian_preturbation!, 0.25)
    replacement         = EO.enclose_replacement(EO.r_NSGA, pop_size)
    termination         = enclose_argument(iteration_termination, pop_size*100)
    
    for i in 1:100
        solution = solvink_hart(objective_function, initialization, selection, crossover, mutation, replacement, termination)
        save(joinpath(@__DIR__, "benchmarks", "constr", "MMOEA", "ez", "problem_"*string(j)*"_"*string(i)*".jld2"), "solution", solution)
    end 
end

In [4]:
for j in 1:4
    problem_set = unsimple_problems
    problem_id = j 
    constraints = problem_set[problem_id].constraints
    pop_size = 100
    dimension = problem_set[problem_id].dimension

    f_pens(x::RealChromosome, constraints::Vector{EO.Constraint}) = sum(EO.Gs(x, constraints))
    objectives = Vector{Any}([problem_set[problem_id].objective])
    for constraint in constraints
        push!(objectives, constraint)
    end
    
    F = typeof(f_pens)
    objective_function  = EO.MultiObjFunction{F}(objectives)
    initialization      = enclose_noargs(interval_real_initialization, dimension, pop_size, objective_function, 0, 100)
    selection           = enclose_arguments(EO.s_tournament, 30, 3)
    crossover           = enclose_arguments(EO.cr_parent_sum, pop_size)
    mutation            = enclose_arguments(gaussian_preturbation!, 0.25)
    replacement         = EO.enclose_replacement(EO.r_NSGA, pop_size)
    termination         = enclose_argument(iteration_termination, pop_size*100)
    
    for i in 1:100
        solution = solvink_hart(objective_function, initialization, selection, crossover, mutation, replacement, termination)
        save(joinpath(@__DIR__, "benchmarks", "constr", "MMOEA", "hard", "problem_"*string(j)*"_"*string(i)*".jld2"), "solution", solution)
    end 
end

### Kim Jong Un

Constraint vialation is being handled by imposing the death penalty.

In [7]:
for j in 1:4
    problem_set = simple_problems
    problem_id = j 
    constraints = problem_set[problem_id].constraints
    pop_size = 100
    dimension = problem_set[problem_id].dimension

    penatly = (x...) -> EO.death_penalty(x..., Inf)     # real numbers led to huge violations, strict measures had to be carried out
    
    objective_function  = problem_set[problem_id].objective
    initialization      = enclose_noargs(interval_real_initialization, dimension, pop_size, objective_function, 0, 100)
    selection           = enclose_arguments(EO.s_tournament, 30, 3)
    crossover           = enclose_arguments(EO.cr_parent_sum, pop_size)
    mutation            = enclose_arguments(gaussian_preturbation!, 0.25)
    replacement         = EO.enclose_replacement(EO.r_keep_best_n, pop_size)
    termination         = enclose_argument(iteration_termination, pop_size*100)
    
    for i in 1:100
        solution = solvink_hart(objective_function, initialization, selection, crossover, mutation, replacement, termination; constraints=constraints, penalty=penatly)
        save(joinpath(@__DIR__, "benchmarks", "constr", "kim", "ez", "problem_"*string(j)*"_"*string(i)*".jld2"), "solution", solution)
    end 
end

In [8]:
for j in 1:4
    problem_set = unsimple_problems
    problem_id = j 
    constraints = problem_set[problem_id].constraints
    pop_size = 100
    dimension = problem_set[problem_id].dimension

    penatly = (x...) -> EO.death_penalty(x..., Inf)     # real numbers led to huge violations, strict measures had to be carried out
    
    objective_function  = problem_set[problem_id].objective
    initialization      = enclose_noargs(interval_real_initialization, dimension, pop_size, objective_function, 0, 100)
    selection           = enclose_arguments(EO.s_tournament, 30, 3)
    crossover           = enclose_arguments(EO.cr_parent_sum, pop_size)
    mutation            = enclose_arguments(gaussian_preturbation!, 0.25)
    replacement         = EO.enclose_replacement(EO.r_keep_best_n, pop_size)
    termination         = enclose_argument(iteration_termination, pop_size*100)
    
    for i in 1:100
        solution = solvink_hart(objective_function, initialization, selection, crossover, mutation, replacement, termination; constraints=constraints, penalty=penatly)
        save(joinpath(@__DIR__, "benchmarks", "constr", "kim", "hard", "problem_"*string(j)*"_"*string(i)*".jld2"), "solution", solution)
    end 
end

NSGA-II with modified tournament operator

In [7]:
for j in 1:4
    problem_set = simple_problems
    problem_id = j 
    constraints = problem_set[problem_id].constraints
    pop_size = 100
    dimension = problem_set[problem_id].dimension

    f_pens(x::RealChromosome, constraints::Vector{EO.Constraint}) = sum(EO.Gs(x, constraints))
    
    F = typeof(enclose_arguments(EO.f_weighted_sum, ones(dimension)))
    objective_function  = EO.MultiObjFunction{F}([problem_set[problem_id].objective, enclose_arguments(f_pens, constraints)])
    initialization      = enclose_noargs(interval_real_initialization, dimension, pop_size, objective_function, 0, 100)
    selection           = enclose_arguments(EO.s_tournament, 30, 3)
    crossover           = enclose_arguments(EO.cr_parent_sum, pop_size)
    mutation            = enclose_arguments(gaussian_preturbation!, 0.25)
    replacement         = EO.enclose_replacement(EO.r_cNSGA, pop_size, constraints)
    termination         = enclose_argument(iteration_termination, pop_size*100)
    
    for i in 1:100
        solution = solvink_hart(objective_function, initialization, selection, crossover, mutation, replacement, termination)
        save(joinpath(@__DIR__, "benchmarks", "constr", "NSGA2", "ez", "problem_"*string(j)*"_"*string(i)*".jld2"), "solution", solution)
    end 
end

In [8]:
for j in 1:4
    problem_set = unsimple_problems
    problem_id = j 
    constraints = problem_set[problem_id].constraints
    pop_size = 100
    dimension = problem_set[problem_id].dimension

    f_pens(x::RealChromosome, constraints::Vector{EO.Constraint}) = sum(EO.Gs(x, constraints))
    
    F = typeof(enclose_arguments(EO.f_weighted_sum, ones(dimension)))
    objective_function  = EO.MultiObjFunction{F}([problem_set[problem_id].objective, enclose_arguments(f_pens, constraints)])
    initialization      = enclose_noargs(interval_real_initialization, dimension, pop_size, objective_function, 0, 100)
    selection           = enclose_arguments(EO.s_tournament, 30, 3)
    crossover           = enclose_arguments(EO.cr_parent_sum, pop_size)
    mutation            = enclose_arguments(gaussian_preturbation!, 0.25)
    replacement         = EO.enclose_replacement(EO.r_cNSGA, pop_size, constraints)
    termination         = enclose_argument(iteration_termination, pop_size*100)
    
    for i in 1:100
        solution = solvink_hart(objective_function, initialization, selection, crossover, mutation, replacement, termination)
        save(joinpath(@__DIR__, "benchmarks", "constr", "NSGA2", "hard", "problem_"*string(j)*"_"*string(i)*".jld2"), "solution", solution)
    end 
end

ASCHEA

In [9]:
for j in 1:4
    problem_set = simple_problems
    problem_id = j 
    constraints = problem_set[problem_id].constraints
    pop_size = 100
    dimension = problem_set[problem_id].dimension

    penatly = EO.p_non_linear   # rg*sum(G(constraints)).^2
    
    objective_function  = problem_set[problem_id].objective
    initialization      = enclose_noargs(interval_real_initialization, dimension, pop_size, objective_function, 0, 100)
    selection           = EO.s_identity
    crossover           = enclose_arguments(EO.cr_ASCHEA, pop_size, 1/3, constraints)
    mutation            = enclose_arguments(gaussian_preturbation!, 0.25)
    replacement         = EO.enclose_replacement(EO.r_ASCHEA, pop_size, 1/3, constraints)
    termination         = enclose_argument(iteration_termination, pop_size*100)
    
    for i in 1:100
        solution = solvink_hart(objective_function, initialization, selection, crossover, mutation, replacement, termination; constraints=constraints, penalty=penatly)
        save(joinpath(@__DIR__, "benchmarks", "constr", "aschea", "ez", "problem_"*string(j)*"_"*string(i)*".jld2"), "solution", solution)
    end 
end

In [10]:
for j in 1:4
    problem_set = unsimple_problems
    problem_id = j 
    constraints = problem_set[problem_id].constraints
    pop_size = 100
    dimension = problem_set[problem_id].dimension

    penatly = EO.p_non_linear   # rg*sum(G(constraints)).^2
    
    objective_function  = problem_set[problem_id].objective
    initialization      = enclose_noargs(interval_real_initialization, dimension, pop_size, objective_function, 0, 100)
    selection           = EO.s_identity
    crossover           = enclose_arguments(EO.cr_ASCHEA, pop_size, 1/3, constraints)
    mutation            = enclose_arguments(gaussian_preturbation!, 0.25)
    replacement         = EO.enclose_replacement(EO.r_ASCHEA, pop_size, 1/3, constraints)
    termination         = enclose_argument(iteration_termination, pop_size*100)
    
    for i in 1:100
        solution = solvink_hart(objective_function, initialization, selection, crossover, mutation, replacement, termination; constraints=constraints, penalty=penatly)
        save(joinpath(@__DIR__, "benchmarks", "constr", "aschea", "hard", "problem_"*string(j)*"_"*string(i)*".jld2"), "solution", solution)
    end 
end

vec aschea

In [11]:
for j in 1:4
    problem_set = simple_problems
    problem_id = j 
    constraints = problem_set[problem_id].constraints
    pop_size = 100
    dimension = problem_set[problem_id].dimension

    penatly = EO.p_non_linear   # rg*sum(G(constraints)).^2
    
    objective_function  = problem_set[problem_id].objective
    initialization      = enclose_noargs(interval_real_initialization, dimension, pop_size, objective_function, 0, 100)
    selection           = EO.s_identity
    crossover           = enclose_arguments(EO.cr_ASCHEA, pop_size, 1/3, constraints)
    mutation            = enclose_arguments(gaussian_preturbation!, 0.25)
    replacement         = EO.enclose_replacement(EO.r_ASCHEA, pop_size, 1/3, constraints)
    termination         = enclose_argument(iteration_termination, pop_size*100)
    
    for i in 1:100
        solution = solvink_hart(objective_function, initialization, selection, crossover, mutation, replacement, termination, "v"; constraints=constraints, penalty=penatly)
        save(joinpath(@__DIR__, "benchmarks", "constr", "aschea_vec", "ez", "problem_"*string(j)*"_"*string(i)*".jld2"), "solution", solution)
    end 
end

In [12]:
for j in 1:4
    problem_set = unsimple_problems
    problem_id = j 
    constraints = problem_set[problem_id].constraints
    pop_size = 100
    dimension = problem_set[problem_id].dimension

    penatly = EO.p_non_linear   # rg*sum(G(constraints)).^2
    
    objective_function  = problem_set[problem_id].objective
    initialization      = enclose_noargs(interval_real_initialization, dimension, pop_size, objective_function, 0, 100)
    selection           = EO.s_identity
    crossover           = enclose_arguments(EO.cr_ASCHEA, pop_size, 1/3, constraints)
    mutation            = enclose_arguments(gaussian_preturbation!, 0.25)
    replacement         = EO.enclose_replacement(EO.r_ASCHEA, pop_size, 1/3, constraints)
    termination         = enclose_argument(iteration_termination, pop_size*100)
    
    for i in 1:100
        solution = solvink_hart(objective_function, initialization, selection, crossover, mutation, replacement, termination, "v"; constraints=constraints, penalty=penatly)
        save(joinpath(@__DIR__, "benchmarks", "constr", "aschea_vec", "hard", "problem_"*string(j)*"_"*string(i)*".jld2"), "solution", solution)
    end 
end